In [25]:
from collections import defaultdict, OrderedDict
from csv import reader
import pandas as pd
import sys
from itertools import chain, combinations
from optparse import OptionParser
from time import perf_counter



# node structure which stores itemname, its frequenct, its parentNode, its children and its next pointer stores the address of 
# the nextnode.

class Node:
    def __init__(self, itemName, frequency, parentNode):
        self.itemName = itemName
        self.count = frequency
        self.parent = parentNode
        self.children = {}
        self.next = None


    


# This function take itemsets and frequency as input and creates a dictionary of frequent items and then creates the fptree
# structure by using nodes and dictionary for storing pointers.

def constructTree(itemSetList, frequency, minSup):
    headerTable = defaultdict(int)
    for idx in range(len(itemSetList)):
        for item in itemSetList[idx]:
            headerTable[item] += frequency[idx]

    headerTable = dict((item, sup) for item, sup in headerTable.items() if sup >= minSup)
    if(len(headerTable) == 0):
        return None, None

    for item in headerTable:
        headerTable[item] = [headerTable[item], None]

    
    fpTree = Node('Null', 1, None)
    for idx in range(len(itemSetList)):
        itemSet=itemSetList[idx]
        itemSet = [item for item in itemSet if item in headerTable]
        itemSet.sort(key=lambda item: headerTable[item][0], reverse=True)
        
        currentNode = fpTree
        for item in itemSet:
            if item in currentNode.children:
                currentNode.children[item].count+=frequency[idx]
            else:
                newItemNode = Node(item, frequency[idx], currentNode)
                currentNode.children[item] = newItemNode

                if(headerTable[item][1] == None):
                    headerTable[item][1] = newItemNode
                else:
                    tempNode = headerTable[item][1]
                   
                    while tempNode.next != None:
                        tempNode = tempNode.next
                    tempNode.next = newItemNode
                    
            currentNode=currentNode.children[item]
            

    return fpTree, headerTable

# this function is used to ascend the nodes of the fp tree.

def ascendFPtree(node, prefixPath):
    if node.parent != None:
        prefixPath.append(node.itemName)
        ascendFPtree(node.parent, prefixPath)

# This function takes the fptree as input and updates the frequent itemset recursively by creating conditional pattern base 
# by ascending nodes of the fp tree by using the frequency map. And recalls the construct tree and mineTree on the conditional
# pattern base that we got.

def mineTree(headerTable, minSup, preFix, freqItemList):
    
    sortedItemList = [item[0] for item in sorted(list(headerTable.items()), key=lambda p:p[1][0])] 
    
    for item in sortedItemList:  
        
        newFreqSet = preFix.copy()
        newFreqSet.add(item)
        freqItemList.append(newFreqSet)
        treeNode = headerTable[item][1] 
        condPats = []
        frequency = []
        while treeNode != None:
            prefixPath = []
            ascendFPtree(treeNode, prefixPath)  

            if len(prefixPath) > 1:
                condPats.append(prefixPath[1:])
                frequency.append(treeNode.count)

            treeNode = treeNode.next  
        conditionalTree, newHeaderTable = constructTree(condPats, frequency, minSup) 
        if newHeaderTable != None:
            mineTree(newHeaderTable, minSup,newFreqSet, freqItemList)

# This function takes frequent itemset as input and return all the rules satisfying the minimum confidence values.

def getRules(freqItemSet, itemSetList, minConf):
    rules = []
    for itemSet in freqItemSet:
        subsets = chain.from_iterable(combinations(itemSet, r) for r in range(1, len(itemSet)))

        count = 0
        for ele in itemSetList:
            if(set(itemSet).issubset(ele)):
                count += 1
        for s in subsets:
            count2 = 0
            for ele in itemSetList:
                if(set(s).issubset(ele)):
                    count2 += 1
            confidence = float(count / count2)
            if(confidence > minConf):
                rules.append([set(s), set(itemSet.difference(s)), confidence])
    return rules

#This function reads the input file and then constructed the fptree from the itemset by calling the function constructTree then
#we called mineTree function which updated frequent item sets and then return the rules by calling getRules function

def fpgrowth(fname, minSupRatio, minConf):
    itemSetList = []
    frequency = []
    
    with open(fname, 'r') as file:
        csv_reader = reader(file)
        
        for line in csv_reader:
            line = list(filter(None, line))
            
            line=line[0].strip(" ").split(" ")
            
            itemSetList.append(line)
            frequency.append(1)
    
    
    minSup = len(itemSetList) * minSupRatio
    fpTree, headerTable = constructTree(itemSetList, frequency, minSup)
    if(fpTree == None):
        
        return [],[]
    else:
        freqItems = []
        mineTree(headerTable, minSup, set(), freqItems)
        rules = getRules(freqItems, itemSetList, minConf)
        return freqItems, rules

if __name__ == "__main__":
    
    inpfile = "dataset.txt"
    minSup = 0.2
    minConf=0.5
    t1_start = perf_counter()
    freqItemSet, rules = fpgrowth(inpfile,minSup,minConf)
    t1_stop = perf_counter()
    print("Elapsed time during the whole program in seconds:",t1_stop-t1_start)
    if len(rules)==0:
        print("No frequent itemset with enough confidence")
    for rule in rules:
        print(rule[0],"---->",rule[1],"with confidence",rule[2])

Elapsed time during the whole program in seconds: 0.0008084000000962988
{'38'} ----> {'39'} with confidence 1.0
{'39'} ----> {'38'} with confidence 0.5454545454545454
{'39'} ----> {'48'} with confidence 0.5454545454545454
{'48'} ----> {'39'} with confidence 0.75


In [26]:
if __name__ == "__main__":
    inpfile = "dataset.txt"
    minSup=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]
    minConf=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]
    print("sup","conf","no. of rules","time")
    
    for j in range(len(minConf)):
        for i in range(len(minSup)):
        
            
            t1_start = perf_counter()
            freqItemSet, rules = fpgrowth(inpfile,minSup[i],minConf[j])
            t1_stop = perf_counter()
            print(minSup[i],minConf[j],len(rules),round((t1_stop-t1_start)*1000,2))
        print()
            

sup conf no. of rules time
0.1 0.1 32 2.5
0.2 0.1 4 1.38
0.3 0.1 0 1.41
0.4 0.1 0 0.83
0.5 0.1 0 0.88
0.6 0.1 0 0.85
0.7 0.1 0 1.28
0.8 0.1 0 1.49
0.9 0.1 0 1.51
1.0 0.1 0 0.75

0.1 0.2 32 2.13
0.2 0.2 4 1.15
0.3 0.2 0 0.91
0.4 0.2 0 0.67
0.5 0.2 0 0.69
0.6 0.2 0 0.48
0.7 0.2 0 0.68
0.8 0.2 0 0.76
0.9 0.2 0 0.64
1.0 0.2 0 0.65

0.1 0.3 29 1.57
0.2 0.3 4 1.0
0.3 0.3 0 1.09
0.4 0.3 0 1.25
0.5 0.3 0 0.74
0.6 0.3 0 0.65
0.7 0.3 0 0.68
0.8 0.3 0 0.65
0.9 0.3 0 0.64
1.0 0.3 0 0.62

0.1 0.4 27 1.57
0.2 0.4 4 0.93
0.3 0.4 0 0.72
0.4 0.4 0 0.68
0.5 0.4 0 0.71
0.6 0.4 0 0.68
0.7 0.4 0 0.64
0.8 0.4 0 0.29
0.9 0.4 0 0.98
1.0 0.4 0 0.65

0.1 0.5 17 1.72
0.2 0.5 4 0.77
0.3 0.5 0 0.51
0.4 0.5 0 0.9
0.5 0.5 0 0.55
0.6 0.5 0 0.76
0.7 0.5 0 0.44
0.8 0.5 0 0.83
0.9 0.5 0 0.67
1.0 0.5 0 0.59

0.1 0.6 15 1.52
0.2 0.6 2 0.84
0.3 0.6 0 0.72
0.4 0.6 0 0.68
0.5 0.6 0 0.66
0.6 0.6 0 0.62
0.7 0.6 0 0.61
0.8 0.6 0 0.62
0.9 0.6 0 0.66
1.0 0.6 0 0.69

0.1 0.7 10 1.67
0.2 0.7 2 0.91
0.3 0.7 0 0.74
0.4 0.7 0 0.71
0.5